### The goal of this notebook is to detemine whether the input is name, number or alphanumeric.

In [ ]:
#Import the libraries
from datetime import datetime
import string
import random
import pandas as pd
import re
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import *
import tensorflow as tf
from tensorflow import keras
import keras
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Dense, Input, Embedding, Dropout, Activation,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import sys
from tensorflow.keras.models import load_model

In [5]:
# Load the data
# 0 = name
# 1 = number
# 2 = alphanumeric
combine=pd.read_csv(r'combine.csv')
combine.drop('Unnamed: 0',axis=1,inplace=True)
combine.head()

,Sample,Label
0,BDMXLHA5M6,2
1,"bana, ish",0
2,FTZLTDVKP7,2
3,103883054743,1
4,MMX244G9L2,2


In [6]:
# input feature
X=combine['Sample']
X=X.to_frame()
X.head()

,Sample
0,BDMXLHA5M6
1,"bana, ish"
2,FTZLTDVKP7
3,103883054743
4,MMX244G9L2


In [7]:
# label
y=combine['Label']
y=y.to_frame()
y.head()

,Label
0,2
1,0
2,2
3,1
4,2


In [8]:
data = y
data = np.array(data)
print(data)
# one hot encode
encoded = to_categorical(data)
print(encoded)

[[2]
 [0]
 [2]
 ...
 [0]
 [1]
 [2]]
[[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [9]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, encoded, test_size=0.1, random_state=42)

In [11]:
X_train['Sample'][237]

'anika dua'

In [12]:
# Tokenizer
max_features = 20000

tokenizer = Tokenizer(num_words=max_features,char_level=True,lower=False)#char_level=True
tokenizer.fit_on_texts(list(X_train['Sample']))
list_tokenized_train=tokenizer.texts_to_sequences(X_train['Sample'])
list_tokenized_test = tokenizer.texts_to_sequences(X_test['Sample'])

In [13]:
list_tokenized_train

[[6, 12, 12, 6, 7, 3, 7, 7, 12, 7, 12, 3],
 [48, 1, 61, 1, 1, 19, 1, 1, 16, 23, 2, 1, 1, 16, 1, 48],
 [2, 25, 25, 38, 4, 40, 8, 24, 52, 41, 50],
 [1, 20, 13, 2, 20, 1, 20, 1],
 [48, 13, 22, 16, 1, 18, 2, 22, 34, 14, 19, 1],
 [62, 1, 22, 13, 16, 2, 17, 13, 14, 56, 15],
 [3, 9, 8, 9, 6, 9, 4, 8, 9, 10, 5, 11],
 [11, 12, 7, 12, 3, 12, 11, 9, 5, 11, 7, 11],
 [1, 56, 1, 16, 59, 1, 20, 23, 2, 1, 13, 17, 15, 59, 1, 16, 36, 1],
 [2, 29, 32, 43, 49, 50, 27, 32, 49, 12, 25],
 [4, 5, 12, 11, 4, 9, 11, 12, 10, 6, 6, 7],
 [11, 5, 4, 4, 3, 8, 6, 7, 5, 12, 12, 7],
 [3, 9, 3, 7, 10, 11, 4, 6, 5, 8, 5, 12],
 [2, 49, 44, 52, 24, 40, 10, 39, 10, 50, 3],
 [58, 16, 1, 55, 15, 1, 22, 1, 16, 2, 55, 1, 22, 17, 15, 13],
 [2, 30, 46, 35, 52, 50, 47, 27, 10, 5, 52],
 [2, 33, 38, 39, 27, 41, 41, 50, 46, 12, 52],
 [2, 25, 44, 41, 5, 37, 33, 41, 31, 10, 29],
 [7, 3, 10, 11, 9, 4, 10, 3, 4, 8, 7, 8],
 [1, 36, 1, 1, 14, 17, 15, 2, 18, 1, 1, 14],
 [8, 12, 10, 3, 10, 12, 9, 3, 5, 4, 12, 7],
 [20, 54, 22, 1, 14, 1, 21, 

In [14]:
# Convert to numpy array
list_tokenized_train = np.asarray(list_tokenized_train)
list_tokenized_test = np.asarray(list_tokenized_test)
y_train = np.asarray(y_train)

In [36]:
# Pad the input feature tokens so that each of them
maxlen = 10
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [37]:
X_t

array([[12,  6,  7, ...,  7, 12,  3],
       [ 1,  1, 16, ..., 16,  1, 48],
       [25, 25, 38, ..., 52, 41, 50],
       ...,
       [ 4,  9, 10, ...,  7,  3,  4],
       [ 7,  8,  3, ..., 12,  8,  5],
       [16, 14, 13, ...,  1, 14, 13]], dtype=int32)

In [38]:
# Embedding Size
embed_size = 100
# Input
inp=Input(shape=(maxlen, ))
# Embedding
x = Embedding(max_features, embed_size)(inp)
# LSTM layer
x = LSTM(20, return_sequences=True,name='lstm_layer')(x)
# Maxpooling layer
x = GlobalMaxPool1D()(x)
# Dense layer
x = Dense(40, activation="relu")(x)
# Dropout layer
x = Dropout(0.1)(x)
# Output layer
x = Dense(3, activation="sigmoid")(x)

In [39]:
model = Model(inputs=inp, outputs=x)

In [40]:
# Compile model
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [41]:
# Train model
batch_size = 128
epochs = 3
model.fit(X_t,y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 21600 samples, validate on 5400 samples
Epoch 1/3
21600/21600 [==============================] - 21s 960us/sample - loss: 0.2252 - accuracy: 0.9740 - val_loss: 0.0077 - val_accuracy: 1.0000
Epoch 2/3
21600/21600 [==============================] - 14s 648us/sample - loss: 0.0040 - accuracy: 1.0000 - val_loss: 0.0010 - val_accuracy: 1.0000
Epoch 3/3
21600/21600 [==============================] - 14s 648us/sample - loss: 0.0010 - accuracy: 1.0000 - val_loss: 3.7235e-04 - val_accuracy: 1.0000


In [42]:
# Save model
model.save("model.h5")

In [43]:
# 0= Name
# 1= Number
# 2= Alphamerical

In [44]:
# Load model
model2 = load_model('model.h5')

In [45]:
# Test the model
z=['184058325']
zz=tokenizer.texts_to_sequences(z)
zzz = pad_sequences(zz, maxlen=maxlen)
xy=model2.predict(zzz)
s=np.argmax(xy,axis=1)
if s==0:
    print('It is a name')
elif s==1:
    print('It is a number')
elif s==2:
    print('It is a alphanumerical')

It is a number
